# ESSE 2220 – Lab 5 Report
## Ultrasonic Distance Logging

**Course:** ESSE 2220  
**Lab Title:** Ultrasonic Distance Logger  
**Date Performed:** 2025-10-24  
**Date Submitted:** 2025-10-24

---
## 1. Names & Group Info
- **Group Number:** 5
- **Members:** Yathharthha Kaushal, Owen

---
## 2. Setup
### 2.1 Circuit Photo

![Circuit Photo](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/circuit.jpeg)

### 2.2 Main Program

In [ ]:
import RPi.GPIO as GPIO #type: ignore
import time
import csv

trigPin = 16
echoPin = 18
MAX_DISTANCE = 220          # define the maximum measuring distance, unit: cm
timeOut = MAX_DISTANCE*60   # calculate timeout according to the maximum measuring distance
fileName = "distance_log.txt"
csvFileName = "distance_log.csv"

def writeToLog(sensorReading, time, iteration):
    # Use try-except to avoid program crash
    try:
        # Print to console
        if(iteration == 1):
            print("time_s    distance_cm")
        print(f"{time:.3f}    {sensorReading:.2f}")
        # Write to text file
        with open(fileName, "a") as f:
            if(iteration == 1):
                f.write("time_s    distance_cm\n")
            f.write(f"{time:.3f}    {sensorReading:.2f}\n")
        # Also write to CSV file
        with open(csvFileName, "a", newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            if(iteration == 1):
                csvwriter.writerow(["time_s", "distance_cm"])
            csvwriter.writerow([f"{time:.3f}", f"{sensorReading:.2f}"])
        return 1
    except:
        return 0

def pulseIn(pin,level,timeOut): # obtain pulse time of a pin under timeOut
    t0 = time.time()
    while(GPIO.input(pin) != level):
        if((time.time() - t0) > timeOut*0.000001):
            return 0
    t0 = time.time()
    while(GPIO.input(pin) == level):
        if((time.time() - t0) > timeOut*0.000001):
            return 0
    pulseTime = (time.time() - t0)*1000000
    return pulseTime
    
def getSonar():     # get the measurement results of ultrasonic module,with unit: cm
    GPIO.output(trigPin,GPIO.HIGH)      # make trigPin output 10us HIGH level 
    time.sleep(0.00001)     # 10us
    GPIO.output(trigPin,GPIO.LOW) # make trigPin output LOW level 
    pingTime = pulseIn(echoPin,GPIO.HIGH,timeOut)   # read plus time of echoPin
    distance = pingTime * 340.0 / 2.0 / 10000.0     # calculate distance with sound speed 340m/s 
    return distance
    
def setup():
    GPIO.setmode(GPIO.BOARD)      # use PHYSICAL GPIO Numbering
    GPIO.setup(trigPin, GPIO.OUT)   # set trigPin to OUTPUT mode
    GPIO.setup(echoPin, GPIO.IN)    # set echoPin to INPUT mode

def loop():
    # Start with iteration 1
    iteration = 1

    # Clear previous log files / create new ones at the start
    with open(fileName, "w") as f:
        f.write("")  # clear the log file at the start
    with open(csvFileName, "w", newline='') as csvfile:
        csvfile.write("")  # clear the CSV log file at the start

    while(True):
        distance = getSonar() # get distance
        # Add delay of 0.1 second between measurements
        time.sleep(0.1)
        # Write distance to log files
        writeToLog(distance, iteration*0.1, iteration)
        # Add iteration count to track time in log
        iteration = iteration + 1
        
if __name__ == '__main__':     # Program entrance
    print ('Program is starting...')
    setup()
    try:
        loop()
    except KeyboardInterrupt:  # Press ctrl-c to end the program.
        GPIO.cleanup()         # release GPIO resource

---
## 3. Observations
### 3.1 Terminal Distance Readings
![Terminal Output](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/terminalOutput.png)

### 3.2 `distance_log.txt` Summary

[![Download TXT File](https://img.shields.io/badge/Download-distance__log.txt-blue?style=for-the-badge&logo=files)](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/labProgram/distance_log.txt)

> The txt file stores the data collected by the sensor and the time, using a new line for each reading, and storing the time at the left, and the distance on the right with 4 spaces. The time interval for readings is 0.100 seconds, and the distance data is stored with two decimal points of precision.

### 3.3 `distance_log.csv` Plot

[![Download CSV File](https://img.shields.io/badge/Download-distance__log.csv-green?style=for-the-badge&logo=microsoftexcel)](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/labProgram/distance_log.csv.xlsx)

![Distance vs Time Graph](https://raw.githubusercontent.com/YK12321/ESSE2220-labs/main/5/graph.png)

### 3.4 Sampling Parameters
- **Sampling Interval (s):** 0.100
- **Total Recording Duration (s):** 13.30s
- Briefly explain how you configured or measured these values.  
> We used a Ultrasonic distance sensor to measure the distance, and we used the time.sleep(0.1) to iterate with a 0.1s interval between sensor readings.

### 3.5 Data Consistency Discussion
- Compare the TXT and CSV outputs. Are they identical or slightly different? Explain why.
> The txt and csv outputs didn't vary a lot, except for the program having to manually add a set number of spaces after storing each data, and adding newlines. For the csv, however, the data was organized into rows/columns.
- Discuss how stable the readings were when the target object was stationary.
> The readings had a smaller variation when the target object was stationary.
- Describe what changed when the object moved faster or slower.
> As the target moved faster, the readings varied a bit more (noticeabloe by small "outelier" peaks before a the peak from a direction switch of the object's velocity.)

---
## 4. Analysis
### 4.1 Echo Pin Behavior
The Echo pin goes HIGH when the ultrasonic pulse is sent and remains HIGH until the reflected pulse returns. The pulse width (duration of the HIGH signal) is proportional to the distance traveled—longer pulse = greater distance.

### 4.2 Distance Calculation Logic
The formula used is: **Distance = (Pulse Duration × Speed of Sound) / 2**

We divide by 2 because the ultrasonic pulse travels to the object and back (round-trip), so the actual distance to the object is half the total distance traveled.

### 4.3 Sources of Error
1. **Temperature variations**: Sound speed changes with temperature (~343 m/s at 20°C), causing distance miscalculations. Appears as systematic offset in readings.
2. **Surface angle/material**: Angled or soft surfaces reflect poorly, causing weak echoes or dropouts. Appears as intermittent invalid readings or large fluctuations.

### 4.4 Error Mitigation Ideas
- **Hardware**: Add a temperature sensor to adjust speed of sound calculations; use voltage divider on Echo pin for protection.
- **Software**: Implement median filtering over multiple readings; reject outliers beyond a threshold; add temperature compensation to distance formula.

### 4.5 Sampling Rate Trade-offs
Shorter delays increase data resolution but may cause issues: sensor needs ~60ms between measurements to avoid echo interference; excessive polling increases CPU load and may yield redundant data if object is stationary. Balance needed between responsiveness and reliability.

### 4.6 Extensions
- **Speed measurement**: Calculate velocity by taking the derivative of distance over time (Δdistance/Δtime) between consecutive readings.
- **Motion detection**: Set a threshold and trigger alerts when distance changes exceed it. Could add an LED or buzzer for visual/audio feedback when motion is detected.

---
## 5. Appendix & Checklist
- [x] Circuit photo inserted and labeled
- [x] Final commented code pasted
- [x] Terminal observations summarized
- [x] TXT and CSV screenshots included
- [x] Distance vs Time graph embedded
- [x] Sampling interval and duration documented
- [x] Analysis questions answered
